In [ ]:
pip install numpy h5py tensorflow

In [ ]:
import numpy as np
import h5py
from tensorflow.keras.datasets import mnist

# Step 1: Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Step 2: Normalize the pixel values (scale to [0, 1])
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Step 3: Reshape data to add a channel dimension (optional, for CNNs)
x_train = np.expand_dims(x_train, axis=-1)  # Shape: (60000, 28, 28, 1)
x_test = np.expand_dims(x_test, axis=-1)    # Shape: (10000, 28, 28, 1)

# Step 4: Save data to an HDF5 file
with h5py.File('mnist_sign_digits.h5', 'w') as h5f:
    h5f.create_dataset('x_train', data=x_train)
    h5f.create_dataset('y_train', data=y_train)
    h5f.create_dataset('x_test', data=x_test)
    h5f.create_dataset('y_test', data=y_test)

print("MNIST dataset has been saved as mnist_sign_digits.h5")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
MNIST dataset has been saved as mnist_sign_digits.h5
